# Test Case LVV-T1600
This case will verify that the integration of the Camera Hexapod with SAL.
The blocks below represent the steps of the test case.

Requirements
* EFD
* Hexapod(s)powered on
* Thermal sensors attached to the six actuators of the hexapod.
* CSC running
* KAFKA producer running

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

In [1]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTHexapod
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

from lsst_efd_client import EfdClient

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.4.1/lib/python3.8/site-packages/jose/backends/cryptography_backend.py:23: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


In [2]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

file:///home/hdrass/WORK/ts_ddsconfig/config/ospl-sp.xml
summit


In [3]:
#This step should fail at NCSA since there is only one domain
print(os.environ["LSST_DDS_DOMAIN_ID"])

11


In [4]:
print(os.environ["LSST_DDS_INTERFACE"])

net1


The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [5]:
#Wait for history data [sec] default value is 60
#os.environ["LSST_DDS_DOMAIN"] = 'citest'
#os.environ["LSST_DDS_HISTORYSYNC"] = "10"

In [6]:
STD_WAIT = 39 # This is the amount of time to wait in between a move just to make sure that the actuators are not overheating

In [7]:
log = logging.getLogger(__name__) # This sets up the logger for the test.
#logging.setLevel(20)
logging.basicConfig(level=logging.INFO)


In [8]:
start_time = datetime.now()
test_message = "Camera Hexapod Integration Test"
#What means index=1 here?
#script = salobj.Controller("Script", index=1)
script = salobj.Controller("Script", index=42658885)
await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
print(f'time to start is {datetime.now() - start_time} [s]')

INFO:Script:Read historical data in 0.00 sec


time to start is 0:00:11.867632 [s]


In [9]:
await script.start_task

This is how you start the remote for the CSC.

In [10]:
# create the remote
#index=1 is the camera hexapod!! index=2 is the M2 hexapod!
csc_index = 1
hexapod_csc = salobj.Remote(name="MTHexapod", domain=script.domain, index=csc_index)

In [11]:
await hexapod_csc.start_task

In [12]:
await hexapod_csc.evt_heartbeat.next(flush=True, timeout=5)

In [50]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY)

[<State.STANDBY: 5>]

In [51]:
await salobj.set_summary_state(hexapod_csc, salobj.State.OFFLINE)

EnabledSubstate.STATIONARY


[<State.STANDBY: 5>, <State.OFFLINE: 4>]

EnabledSubstate.STATIONARY


In [45]:
await salobj.set_summary_state(hexapod_csc, salobj.State.DISABLED)

EnabledSubstate.STATIONARY


[<State.ENABLED: 2>, <State.DISABLED: 1>]

In [32]:
#This is to bring the state machine in the right starting state
await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED)

[<State.ENABLED: 2>]

In [33]:
print(hexapod_csc)
#Bo:Check which hexapod we are controlling
whichHex1 = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("Hexapod hex", whichHex1.MTHexapodID)
print(hexapod_csc.salinfo.index)

Hexapod hex 1
1


In [19]:
#This cell makes sure that the data go to EFD
#may need to wait a few seconds before event shows up in EFD
#client = EfdClient('ncsa_teststand_efd')
client = EfdClient('summit_efd')

In [47]:
end = Time(datetime.now())
start = end - timedelta(hours=3)
dfe = await client.select_time_series('lsst.sal.MTHexapod.logevent_summaryState', '*', start.tai, end.tai, csc_index)

In [48]:
dfe

,MTHexapodID,priority,private_host,private_identity,private_kafkaStamp,private_origin,private_rcvStamp,private_revCode,private_seqNum,private_sndStamp,summaryState
2021-03-05 14:17:19.779000+00:00,1,0,0,MTHexapod:1,1.614954e+09,5360,1.614954e+09,959a0a03,2,1.614954e+09,3
2021-03-05 14:19:16.239000+00:00,1,0,0,MTHexapod:1,1.614954e+09,5360,1.614954e+09,959a0a03,3,1.614954e+09,5
2021-03-05 14:21:30.557000+00:00,1,0,0,MTHexapod:1,1.614954e+09,5360,1.614954e+09,959a0a03,4,1.614954e+09,1
2021-03-05 14:21:42.257000+00:00,1,0,0,MTHexapod:1,1.614954e+09,5360,1.614954e+09,959a0a03,5,1.614954e+09,3
2021-03-05 14:50:17.989000+00:00,1,0,0,MTHexapod:1,1.614956e+09,5360,1.614956e+09,959a0a03,6,1.614956e+09,5
2021-03-05 14:53:40.977000+00:00,1,0,0,MTHexapod:1,1.614956e+09,5360,1.614956e+09,959a0a03,7,1.614956e+09,1
2021-03-05 14:53:55.960000+00:00,1,0,0,MTHexapod:1,1.614956e+09,5360,1.614956e+09,959a0a03,8,1.614956e+09,2
2021-03-05 15:26:22.586000+00:00,1,0,0,MTHexapod:1,1.614958e+09,5360,1.614958e+09,959a0a03,9,1.614958e+09,1


In [22]:
def enabled_substate_callback(evt):
    """Print the enabled substate when event is received."""
    print(MTHexapod.EnabledSubstate(evt.enabledSubstate))

In [23]:
def in_position_callback(evt):
    """Print the in position event when it is received."""
    print(evt.in_position)

This next block will make sure that the CSC is ready for the test

In [34]:
connected = await hexapod_csc.evt_connected.aget() # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
#commandable = connected.command
telemetry_working = connected.telemetry

if not connected or not telemetry_working:
    raise Exception("Hexapod not connected or telemetry not being received.") 

In [35]:
controller_state = await hexapod_csc.evt_controllerState.aget()
state = controller_state.controllerState
offline_substate = controller_state.offlineSubstate
print(state)
#This is only for the real system. The simulater only has the AVAILABLE offline state 

'''  
if not state == salobj.State.OFFLINE and not offline_substate == MTHexapod.OfflineSubstate.AVAILABLE:
    raise Exception("Controller must be changed to Available Offline Substate.")
'''
commandable_by_dds = await hexapod_csc.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")

2


The next four blocks bring the CSC to the Enabled state.

In [ ]:
await hexapod_csc.cmd_enterControl.set_start() # Send the enterControl command notice the casing and the set_start method

In [ ]:
await hexapod_csc.cmd_start.set_start() 

In [ ]:
await hexapod_csc.cmd_enable.set_start()

In [ ]:
await hexapod_csc.cmd_clearError.set_start() # This clears the error

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
# Step skiped see deviation LVV-T1802 Test Step 5                                  
#hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
#await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900, u=0.12,v=0,w=0,sync=True)



In [36]:
#Command used to stop the hexapod 
await hexapod_csc.cmd_stop.set_start()

In [43]:
#This command is to set the Hexapod to zero position
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [44]:
script.log.info(f"TEST test")

INFO:Script:TEST test


In [41]:
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 7")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

INFO:Script:START- Camera Hexapod Integration Test -- LVV-T1600 Test Step 7


hex position
     -0.93        0.19       -0.23        0.00        0.00        0.00  



In [42]:
hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
await hexapod_csc.cmd_move.set_start(x=500,y=-500,z=200, u=0.01,v=-0.015,w=0,sync=True)
await asyncio.sleep(15.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 7")

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


INFO:Script:STOP- Camera Hexapod Integration Test -- LVV-T1600 Test Step 7


hex position
    500.18     -501.11      199.99        0.01       -0.02       -0.00  



In [43]:
#This command is to set the Hexapod to zero position
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
# thermal sensors

This block moves the hexapod to a large position but stops it after waiting 3 seconds

In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 11")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=5000,u=0,v=0,w=0,sync=True)
await asyncio.sleep(3)
await hexapod_csc.cmd_stop.set_start()

await asyncio.sleep(15.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')


#cs = await hexapod_csc.evt_controllerState.aget()
#print(MTHexapod.EnabledSubstate(cs.enabledSubstate))
#if cs.enabledSubstate != MTHexapod.EnabledSubstate.CONTROLLED_STOPPING:
#     raise Exception(f"Controller's substate is not {MTHexapod.EnabledSubstate.CONTROLLED_STOPPING}. It is {MTHexapod.EnabledSubstate(cs.enabledSubstate)}")

# cs = await hexapod_csc.evt_controllerSubstate.aget()
# print(MTHexapod.EnabledSubstate(cs.enabledSubstate))
# if cs.enabledSubstate != MTHexapod.EnabledSubstate.STATIONARY:
#     raise Exception(f"Controller's substate is not {MTHexapod.EnabledSubstate.STATIONARY}. It is {MtHexapod.EnabledSubstate(cs.enabledSubstate)}")

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 11")

In [ ]:
#Step 15
await asyncio.sleep(STD_WAIT)

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
await asyncio.sleep(STD_WAIT)

This block moves the hexapod via the lookup table:

In [ ]:
#This command is obsolete
#await hexapod_csc.cmd_moveLUT.set_start(az=180,elev=60, temp=10)

script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 17")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
#Switch CompensationMode on
await hexapod_csc.cmd_setCompensationMode.set_start(enable=True)

lutMode = await hexapod_csc.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lutMode.enabled)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
await asyncio.sleep(10.)

In [ ]:
#move the hexapod Not allowed here! First leave compensation mode than move to new postion!!! Otherwise the you hexpod gets stuck in compensation  mode
#await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)

#await asyncio.sleep(10.)

In [ ]:
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

#Switch CompensationMode off 
await hexapod_csc.cmd_setCompensationMode.set_start(enable=False)
lutMode = await hexapod_csc.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lutMode.enabled)

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 17")

In [ ]:
await hexapod_csc.cmd_setCompensationMode.set_start(enable=False)


In [ ]:
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
posC = await hexapod_csc.evt_compensatedPosition.aget(timeout=10.)
print('Compensated position')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posC, i), end= ' ')
print('\n')

posU = await hexapod_csc.evt_uncompensatedPosition.aget(timeout=10.)
print('Uncompensated position')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posU, i), end= ' ')
print('\n')

Analysis of the LUT correction

In [ ]:
LUTfile = '%s/notebooks/ts_config_mttcs/MTHexapod/v1/default.yaml'%(os.environ["HOME"])
print(LUTfile)
with open(LUTfile, 'r') as stream:
    aa = yaml.safe_load(stream)
if hexapod_csc.salinfo.index == 1:
    elevCoeff = aa['camera_config']['elevation_coeffs']
    tCoeff = aa['camera_config']['temperature_coeffs']
elif hexapod_csc.salinfo.index == 2:
    elevCoeff = aa['m2_config']['elevation_coeffs']
    tCoeff = aa['m2_config']['temperature_coeffs']

In [ ]:
pred = []
elev=89.999
print('Predicted LUT compensation:')
for i in range(6):
    coeff = elevCoeff[i] #starts with C0
    mypoly = np.poly1d(coeff[::-1]) #poly1d needs C5 first
    pred.append(mypoly(elev))
    print('%10.2f '%pred[i], end= ' ')
print('\n')
print('Actual LUT compensation:')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posC, i), end= ' ')
print('\n')

In [ ]:
#Step 22
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 24")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
await hexapod_csc.cmd_move.set_start(x=500,y=800,z=200,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

In [ ]:
await hexapod_csc.cmd_offset.set_start(x=0,y=0,z=500,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 24")


# cs = await MThexapod_csc.evt_controllerState.aget()
# if cs.enabledSubstate != MThexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT:
#     raise Exception(f"Controller's substate is not {MThexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT}. It is {MThexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# cs = await MThexapod_csc.evt_controllerSubstate.aget()
# if cs.enabledSubstate != MThexapod.SetEnabledSubstateParam.STATIONARY:
#     raise Exception(f"Controller's substate is not {MThexapod.SetEnabledSubstateParam.STATIONARY}. It is {MThexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# in_position = await MThexapod_csc.evt_inPosition.aget(timeout=10)
# if in_position.inPosition is False:
#     raise Exception("Controller not in position.")

In [ ]:
#Step 27
await asyncio.sleep(STD_WAIT)

In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1600 Test Step 29")#Pivot test
#Step 29
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')


In [ ]:
#Step 30 -- set a new Pivot point
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=0)
await asyncio.sleep(10.)

In [ ]:
#Step 31
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
await asyncio.sleep(10.)

In [ ]:
#Set the pivot point back to the original value (0.703m in um)
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=703000)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
script.log.info(f"STOP- {test_message} -- LVV-T1600 Test Step 29")

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
script.log.info(f"START- {test_message} -- LVV-T1600 Endurance Test")
for i in range(100):
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)


    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)


script.log.info(f"STOPP- {test_message} -- LVV-T1600 Endurance Test ")

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY) # Transition the CSC to Standby state

In [ ]:
await domain.close() # Close the remote connection